# Solving the Royal Tour Problem with Minimax
The Royal Tour is a series of moves starting with the position below, white to move. White can force black to take all of the white's pieces except one, and then win the game with a 9 piece jump.

It is an interesting problem because a search tree must go at least 13 levels deep to 'see' the victory from the start.

0.          [FEN "W:W27,19,18,11,7,6,5:B28,26,25,20,17,10,9,4,3,2"]
1. 19-15-->	[FEN "B:W27,18,15,11,5,6,7:B25,26,28,17,20,9,10,2,3,4"]
2. 10x19-->	[FEN "W:W27,18,11,5,6,7:B25,26,28,17,19,20,9,2,3,4"]
3. 5-1-->	[FEN "B:W27,18,11,6,7,K1:B25,26,28,17,19,20,9,2,3,4"]
4. 3x10-->	[FEN "W:W27,18,11,6,K1:B25,26,28,17,19,20,9,10,2,4"] 
5. 11-8-->	[FEN "B:W27,18,6,8,K1:B25,26,28,17,19,20,9,10,2,4"]
6. 4x11-->	[FEN "W:W27,18,6,K1:B25,26,28,17,19,20,9,10,11,2"]
7. 27-24-->	[FEN "B:W24,18,6,K1:B25,26,28,17,19,20,9,10,11,2"]
8. 20x27-->	[FEN "W:W18,6,K1:B25,26,27,28,17,19,9,10,11,2"]
9. 18-14-->	[FEN "B:W14,6,K1:B25,26,27,28,17,19,9,10,11,2"]
10. 9x18-->	[FEN "W:W6,K1:B25,26,27,28,17,18,19,10,11,2"]
11. 1-5-->	[FEN "B:WK5,6:B25,26,27,28,17,18,19,10,11,2"]
12. 2x9-->	[FEN "W:WK5:B25,26,27,28,17,18,19,9,10,11"]
13. 5x32-->	[FEN "B:WK32:B28"]

# MinmaxA: First implementation of minimax algorithm

This [Linkedin Learning class](https://www.linkedin.com/learning/ai-algorithms-for-gaming?u=73725428) got me started on minimax algoritm. 

MinmaxA uses a minimax algorithm to traverse a node tree for all legal moves to a specified depth. At each node, it uses the internal functions of board2.Board to find legal moves and to make the next move for the next level of the tree. For each node, it makes a deep copy of the Board instance before passing it to the next level.

Using copy.deepcopy for each Board object is *very* slow.

> **A note about Python versions.** MinmaxA was originally developed using Python 3.8-32 bit. Jupyter notebooks requires 64 bit version of Python. So, these tests are run on Python 3.10. Running 64 bit python carries a significant speed benefit:

>From the same position with 32 Python:  
>MinMaxA@d5 to move as White.  
>MinMaxA@d5 - White's Move: [18, 14]  
Score: -3; Time: 6.06; Nodes: 15047; nps: 2483)  
>2483 nps with 32 bit v 3661 nps with 64 bit, a 33% speed improvement

## Ideas for Making MinmaxA faster:

- More efficient method for move generation, bitboards maybe. [Checker Bitboards Tutorial](https://www.3dkingdoms.com/checkers/bitboards.htm) by Jonathan Kreuzer.
- Multi-threading to work on more than 1 branch of the tree at a time.
- Implement tree pruning.

In [1]:
from board2 import Board
import engines
from positions import positions
from debug_pos import Debug_pos

pos = '[FEN "W:W27,19,18,11,7,6,5:B28,26,25,20,17,10,9,4,3,2"]'
b = Board(pos)
engine = engines.minmaxA(b, maxdepth=5)
db = Debug_pos(b, engine)
print(b.printBoard())
move, engine = db.debug()
print(db.getEngineInfo(move))


[FEN "W:W27,18,19,11,5,6,7:B25,26,28,17,20,9,10,2,3,4"]
    -----------------
 1 |   -   b   b   b | 4
 5 | w   w   w   -   | 8
 9 |   b   b   w   - | 12
13 | -   -   -   -   | 16
17 |   b   w   w   b | 20
21 | -   -   -   -   | 24
25 |   b   b   w   b | 28
29 | -   -   -   -   | 32
    -----------------
MinMaxA@d5 to move as White.
MinMaxA@d5 - White's Move: [18, 14]
Score: -3; Time: 4.11; Nodes: 15047; nps: 3661)


## MinimaxA: The Royal Tour Test

The Royal Tour Test is designed to measure how close an engine is to solving the Royal Tour Problem. It starts at the last position of the Royal Tour, after white has already won. The engine is asked to search one level deep. Since there are no legal moves, it finds the win immediately. Then the engine is given the next to the last position and asked to search two levels deep. Again, the engine will definitely find the win that is only one move away. The process continues, the positions getting closer to the starting position, and the engines being asked to search more deeply. The process stops if any level takes more than 5 minutes, 300 seconds, to finish.

MinmaxA reached level 10, but it was over the time limit by a whopping 43 min 30 sec. Plainly, asking MinMaxA for a deeper search is unrealistic.

In [4]:
import engines
import royalTourTest as rt

rt.run(engines.minmaxA)

MinMaxA@d1 as Black moves None
	Score: -100, Nodes: 0, NPS: 0 Error, Time: 0.0
MinMaxA@d2 as White moves [5, 14, 23, 16, 7, 14, 21, 30, 23, 32]
	Score: 100, Nodes: 37, NPS: 0 Error, Time: 0.0
MinMaxA@d3 as Black moves [2, 9]
	Score: -100, Nodes: 38, NPS: 1900, Time: 0.02
MinMaxA@d4 as White moves [1, 5]
	Score: 100, Nodes: 39, NPS: 1950, Time: 0.02
MinMaxA@d5 as Black moves [9, 18]
	Score: -100, Nodes: 40, NPS: 2000, Time: 0.02
MinMaxA@d6 as White moves [18, 14]
	Score: 100, Nodes: 3357, NPS: 3949, Time: 0.85
MinMaxA@d7 as Black moves [20, 27]
	Score: -100, Nodes: 3358, NPS: 3859, Time: 0.87
MinMaxA@d8 as White moves [27, 24]
	Score: 100, Nodes: 142643, NPS: 3839, Time: 37.15
MinMaxA@d9 as Black moves [4, 11]
	Score: -100, Nodes: 142644, NPS: 3888, Time: 36.68
MinMaxA@d10 as White moves [27, 24]
	Score: 100, Nodes: 10411467, NPS: 3572, Time: 2914.46
TimeOut


# MinMaxB Is Much Faster
MinMaxB improves on MinMaxA by avoiding the deep copy of the board2.Board object at each tree node. Instead, MinMaxB passes along a copy of Board.position along with Board.onMove, to each node. The next node implants the position data into a single instance of a Board object to perform the necessary move generation and position update. (Board.position is a 46 element list containing the location of pieces on the board.) Using the same Board object for each node instead of a copying a node object for each node save an enormous amount of time.

MinMaxB processes the same position and depth at 14195 nps vs MinMaxA's 3661 nps - 3.9 times faster. In practice MinMaxB is usually 5 times faster.

In [2]:
from board2 import Board
import engines
from positions import positions
from debug_pos import Debug_pos

pos = '[FEN "W:W27,19,18,11,7,6,5:B28,26,25,20,17,10,9,4,3,2"]'
b = Board(pos)
engine = engines.minmaxB(b, maxdepth=5)
db = Debug_pos(b, engine)
print(b.printBoard())
move, engine = db.debug()
print(db.getEngineInfo(move))


[FEN "W:W27,18,19,11,5,6,7:B25,26,28,17,20,9,10,2,3,4"]
    -----------------
 1 |   -   b   b   b | 4
 5 | w   w   w   -   | 8
 9 |   b   b   w   - | 12
13 | -   -   -   -   | 16
17 |   b   w   w   b | 20
21 | -   -   -   -   | 24
25 |   b   b   w   b | 28
29 | -   -   -   -   | 32
    -----------------
MinMaxB@d5 to move as White.
MinMaxB@d5 - White's Move: [18, 14]
Score: -3; Time: 4.17; Nodes: 15047; nps: 3608)


## MinMaxB: The Royal Tour Test
Like MinMaxA, MinMaxB also got to level 10. But MinMaxB went over the 300 second deadline by only 130 seconds. This compared to MinMaxA missing the deadline by 2600 seconds.

Level 9 has 142,000 nodes. Level 10 has nearly 10.5 million nodes. Minmaxb is almost over the hump, ready to move on to depth 11. MinmaxA is not even close. Perhaps adding pruning will get us close.

In [2]:
import engines
import royalTourTest as rt

rt.run(engines.minmaxB)

MinMaxB@d1 as Black moves None
	Score: -100, Nodes: 0, NPS: 0 Error, Time: 0.0
MinMaxB@d2 as White moves [5, 14, 23, 16, 7, 14, 21, 30, 23, 32]
	Score: 100, Nodes: 37, NPS: 0 Error, Time: 0.0
MinMaxB@d3 as Black moves [2, 9]
	Score: -100, Nodes: 38, NPS: 0 Error, Time: 0.0
MinMaxB@d4 as White moves [1, 5]
	Score: 100, Nodes: 39, NPS: 1950, Time: 0.02
MinMaxB@d5 as Black moves [9, 18]
	Score: -100, Nodes: 40, NPS: 0 Error, Time: 0.0
MinMaxB@d6 as White moves [18, 14]
	Score: 100, Nodes: 3357, NPS: 20981, Time: 0.16
MinMaxB@d7 as Black moves [20, 27]
	Score: -100, Nodes: 3358, NPS: 18655, Time: 0.18
MinMaxB@d8 as White moves [27, 24]
	Score: 100, Nodes: 142643, NPS: 23006, Time: 6.2
MinMaxB@d9 as Black moves [4, 11]
	Score: -100, Nodes: 142644, NPS: 23616, Time: 6.04
MinMaxB@d10 as White moves [27, 24]
	Score: 100, Nodes: 10411467, NPS: 24165, Time: 430.84
TimeOut


# MinMaxAB: Alpha-beta pruning solves the Royal Tour Problem in Under 2 Minutes!

(This is the same engine at MinMaxB, but with ab = True.)

At depth=10, MinMaxB searched 10,411,467 nodes in 431 seconds. Pruning reduced the number of nodes at that depth to 123,554 completed in 4.49 seconds! A staggering increase in efficiency!

In [1]:
import engines
import royalTourTest as rt

rt.run(engines.minmaxB)

MinMaxB@d1 as Black moves None
	Score: -100, Nodes: 0, NPS: 0 Error, Time: 0.0
MinMaxB@d2 as White moves [5, 14, 23, 16, 7, 14, 21, 30, 23, 32]
	Score: 100, Nodes: 23, NPS: 0 Error, Time: 0.0
MinMaxB@d3 as Black moves [2, 9]
	Score: -100, Nodes: 24, NPS: 0 Error, Time: 0.0
MinMaxB@d4 as White moves [1, 5]
	Score: 100, Nodes: 25, NPS: 1250, Time: 0.02
MinMaxB@d5 as Black moves [9, 18]
	Score: -100, Nodes: 26, NPS: 0 Error, Time: 0.0
MinMaxB@d6 as White moves [18, 14]
	Score: 100, Nodes: 59, NPS: 0 Error, Time: 0.0
MinMaxB@d7 as Black moves [20, 27]
	Score: -100, Nodes: 60, NPS: 3000, Time: 0.02
MinMaxB@d8 as White moves [27, 24]
	Score: 100, Nodes: 4895, NPS: 17482, Time: 0.28
MinMaxB@d9 as Black moves [4, 11]
	Score: -100, Nodes: 4896, NPS: 22254, Time: 0.22
MinMaxB@d10 as White moves [27, 24]
	Score: 100, Nodes: 123554, NPS: 27517, Time: 4.49
MinMaxB@d11 as Black moves [3, 10]
	Score: -100, Nodes: 123555, NPS: 27335, Time: 4.52
MinMaxB@d12 as White moves [5, 1]
	Score: 100, Nodes: 282